In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import modulo as md

In [3]:
headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}

In [4]:
url='https://fbref.com/es/equipos/b8fd03ef/2022-2023/all_comps/Estadisticas-de-Manchester-City-Todas-las-competencias'
res = requests.get(url, headers=headers)
soup=BeautifulSoup(res.text, 'html.parser')


# Aca esta la funcion unificada

In [7]:
def get_players_mean(res,url):# ejemplo url https://fbref.com/es/equipos/b8fd03ef/2022-2023/all_comps/Estadisticas-de-Manchester-City-Todas-las-competencias
    
    codigoE=url.split('equipos/')[1].split('/20')[0]
    soup=BeautifulSoup(res.text, 'html.parser')
    pagina=pd.read_html(url)
    tamaño=len(soup.find('div', class_='filter switcher'))
    tablas=[]
    tablas.append(pagina[0])

    ##BLOQUE 1 

    #defino que tablas voy a usar
    for i in range(1,5):
        tablas.append(pagina[(i*tamaño-(i-1))])
    for i in range(6,9):
        tablas.append(pagina[(i*tamaño-(i-1))])
    tablas.append(pagina[10*tamaño-9])

    #elimino los encabezados de las tablas y filtro a los jugadores con mas de 5 min jugados en promedio en la temporada
    for i in range(9):
        tablas[i].columns=tablas[i].columns.droplevel(level=0)
        tablas[i]=tablas[i][tablas[i]['90 s']>=5]

    
    ## BLOQUE 2
    
    dfs_arreglados=[]
    for i, df in enumerate(tablas):

        if i == 0:
            #elimino las columnas duplicadas
            duplicates=['G-TP','npxG','xAG','npxG+xAG']
            for duplicate in duplicates:
                df=md.drop_duplicates_columns(df,duplicate,1)
            #me quedo con las columnas que necesito
            df = df.loc[:,['Jugador', 'Posc', 'Edad', '90 s', 'PrgC', 'PrgP', 'PrgR', 'Ast','G-TP', 'npxG', 'xAG', 'npxG+xAG']]
            # como los apartados de Prg estan en total y no por 90min los pasare a 90 min
            Prgs=['PrgC','PrgP','PrgR']   
            for Prg in Prgs:
                df[f'{Prg}']=round(df[f'{Prg}']/df['90 s'], 2)
            #agrego el dataframe nuevo a la lista de dfs 
            dfs_arreglados.append(df)

        if i ==1:
            #me quedo con las columnas que necesito
            df = df.loc[:, ['Jugador','GC90','% Salvadas','PaC%',]]
            #agrego el dataframe nuevo a la lista de dfs
            dfs_arreglados.append(df)

        if i ==2:    
            
            df=md.drop_duplicates_columns(df,'Long. prom.',1)
            df=md.drop_duplicates_columns(df,'Int.',2)
            df=md.drop_duplicates_columns(df,'%deLanzamientos',0)
            df['Int.']=round(df['Int.']/df['90 s'],2)
            #me quedo con las columnas que necesito
            df = df.loc[:, ['Jugador', 'PSxG/SoT', '/90','%deLanzamientos','Int.','% de Stp', 'Núm. de OPA/90', 'DistProm.', 'Long. prom.']]
            #agrego el dataframe nuevo a la lista de dfs 
            dfs_arreglados.append(df)
        if i == 3:
            #cambio el apartado de Dis a Dis por 90min
            df.loc[:, 'Dis'] = round(df['Dis'] / df['90 s'], 2)
            #me quedo con las columnas que necesito
            df = df.loc[:, ['Jugador','Dis']]
            #agrego el dataframe nuevo a la lista de dfs
            dfs_arreglados.append(df)
        if i == 4:
            #elimino las columnas duplicadas
            duplicates=['Int.','% Cmp']
            for duplicado in duplicates:
                df=md.drop_duplicates_columns(df,duplicado,0)
            df['Int.']=round(df['Int.']/df['90 s'],2)
            #me quedo con las columnas que necesito
            df = df.loc[:, ['Jugador','Int.','% Cmp']]
            #agrego el dataframe nuevo a la lista de dfs
            dfs_arreglados.append(df)

        if i == 5:

            df.loc[:,'ACT']=round(df['ACT']/df['90 s'],2)
            #me quedo con las columnas que necesito
            df = df.loc[:, ['Jugador','ACT']]
            #agrego el dataframe nuevo a la lista de dfs
            dfs_arreglados.append(df)

        if i == 6:

            df=md.drop_duplicates_columns(df,'Tkl',0)
            df['Tkl']=round(df['Tkl']/df['90 s'], 2)
            df['Bloqueos']=round(df['Bloqueos']/df['90 s'], 2)
            df['Int']=round(df['Int']/df['90 s'], 2)
            df['Desp.']=round(df['Desp.']/df['90 s'], 2)
            #me quedo con las columnas que necesito
            df = df.loc[:, ['Jugador','Desp.','Int','Bloqueos','Tkl']]
            #agrego el dataframe nuevo a la lista de dfs
            dfs_arreglados.append(df)

        if i == 7:
            df.loc[:,'Ataq. pen.']=round(df['Ataq. pen.']/df['90 s'], 2)
            df.loc[:,'Toques']=round(df['Toques']/df['90 s'], 2)
            df.loc[:,'Succ']=round(df['Succ']/df['90 s'], 2)
            #me quedo con las columnas que necesito
            df = df.loc[:, ['Jugador','Ataq. pen.','Toques','Succ']]
            #agrego el dataframe nuevo a la lista de dfs
            dfs_arreglados.append(df)

        if i == 8:

            df.loc[:,'Ganados']=round(df['Ganados']/df['90 s'], 2)
            #me quedo con las columnas que necesito
            df = df.loc[:, ['Jugador','Ganados']]
            #agrego el dataframe nuevo a la lista de dfs
            dfs_arreglados.append(df)

    ## BLOQUE 3
    temp=dfs_arreglados[0]
    i=0
    while i < 8:
        resultado = pd.merge(temp,dfs_arreglados[i+1], on='Jugador', how='outer')
        temp=resultado
        i+=1
    ## BLOQUE 4
    allteams=pd.read_csv(r'C:\Users\ARTUROS\Desktop\Archivos Adrian\personal\proyectos personales\lacaprichosa\dataset\All_teams.csv')
    resultado['Jugador']=int(allteams[allteams['id Web']==f'{codigoE}']['id Local'].iloc[0])

    ## BLOQUE 5
    resultado=resultado.drop('Posc', axis=1)
    resultado=resultado.groupby('Jugador').mean().reset_index()
    resultado=resultado.rename(columns={'Jugador':'Equipo id'})

    return resultado


In [4]:
equipito=md.get_players_mean(res,'b8fd03ef',url)
print((equipito.columns))
equipito

Index(['Equipo id', 'Edad', '90 s', 'PrgC', 'PrgP', 'PrgR', 'Ast', 'G-TP',
       'npxG', 'xAG', 'npxG+xAG', 'GC90', '% Salvadas', '% Salvadas', 'PaC%',
       'PSxG/SoT', '/90', '%deLanzamientos', 'Int._x', '% de Stp',
       'Núm. de OPA/90', 'DistProm.', 'Long. prom.', 'Dis', 'Int._y', '% Cmp',
       'ACT', 'Desp.', 'Int', 'Bloqueos', 'Tkl', 'Ataq. pen.', 'Toques',
       'Succ', 'Ganados'],
      dtype='object')


,Equipo id,Edad,90 s,PrgC,PrgP,PrgR,Ast,G-TP,npxG,xAG,...,% Cmp,ACT,Desp.,Int,Bloqueos,Tkl,Ataq. pen.,Toques,Succ,Ganados
0,12,26.090909,30.381818,1.971364,3.899091,3.891364,0.141818,0.168636,0.152273,0.127727,...,86.559091,2.050455,0.804091,0.636818,0.657273,0.949091,2.315,58.028636,0.732273,0.804091


esta es la lista de columnas descrita en la documentacion

In [88]:
['Goles sin penalización',  'Goles esperados (xG) sin contar penaltis',  'Asistencias',  'xAG',  'npxG + xAG',
'Pases progresivos',  'Acarreos progresivos',  'Pases progresivos Rec',  ' Goles en contra',  'Porcentaje de salvadas',
'% Salvadas (Tiros penales)',  'Porcentaje de portería a cero',  'PSxG-GA, PSxG/SoT',  'Lanzamiento %',  'Saques de meta', 
'Prom. de longitud de los saques de meta',  'Acciones defensivas fuera del área penal',  'Distancia promedio de las acciones defensivas',
'% de cruces detenidos', 'Total de disparos', 'Pases intentados',  '% de pase completo',  'Acciones para la creación de tiros',
'Derribos',  'Intercepciones',  'Bloqueos',  'Despeje',  'Toques (Ataq. pen.)',  'Tomas exitosas',  'Toques',  'Aéreos ganados']

31

y esta es la misma lista pero con los nombres que toman en la tabla

In [ ]:
['G-TP', 'npxG',  'Ast',  'xAG',  'npxG + xAG', 'PrgP',  'PrgC',  'PrgR',  'GC90',  '% Salvadas',
'% Salvadas',  'PaC%',  '/90', 'PSxG/SoT',  'Lanzamiento %', 'Int._x', 'Long. prom.',  'Núm. de OPA/90',  'DistProm.',
'% de Stp',  'Dis', 'Int._y',  '% Cmp',  'ACT', 'Tkl',  'Int',  'Bloqueos',  'Desp.',  'Ataq. pen.',  'Succ',  'Toques',  'Ganados']

arqueros=pagina[8].head(2)
pagina[0].columns=pagina[0].columns.droplevel(level=0)
df_jugadores=pagina[0]
df_jugadores[df_jugadores['Posc']!='PO']
df_jugadores[df_jugadores['Titular']>=5]
tablaprueba=tablaprueba.drop(tablaprueba.columns[indice],axis=1)
df_jugadores

# Aca esta la logica detras de cada bloque de la funcion

## bloque 1: selector de tablas

In [ ]:
#aqui estaba calculando la sucesion para poder elegir correctamente que tablas
[0, 7, 13, 19,25,37,43,49,61]
[0, 1*tamaño-0, 2*tamaño-1,3*tamaño-2,4*tamaño-3,   6*tamaño-5,7*tamaño-6,8*tamaño-7,    10*tamaño-9 ]

elige tablas tablas[#]

In [ ]:
tamaño=len(soup.find('div', class_='filter switcher'))
tablas=[]
tablas.append(pagina[0])
for i in range(1,5):
    tablas.append(pagina[(i*tamaño-(i-1))])
for i in range(6,9):
    tablas.append(pagina[(i*tamaño-(i-1))])
tablas.append(pagina[10*tamaño-9])


for i in range(9):
    tablas[i].columns=tablas[i].columns.droplevel(level=0)
    tablas[i]=tablas[i][tablas[i]['90 s']>=5]

    

## bloque 2: For principal

elimina sobrantes tablas[0]
cambia las progresiones a por 90s tablas[0]

In [ ]:
duplicates_a_eliminar=['G-TP','npxG','xAG','npxG+xAG']
columnas_eliminar0=['País','PJ','Titular','Mín','Partidos','Ass', 'Gls.', 'G+A','TP','TPint','TA','TR','xG']
tablas[0]=tablas[0].drop(columnas_eliminar0, axis=1)
for duplicate in duplicates_a_eliminar:
    tablas[0]=md.drop_duplicates_columns(tablas[0],duplicate,1)
Prgs=['PrgC','PrgP','PrgR']
for Prg in Prgs:
    tablas[0][f'{Prg}']=round(tablas[0][f'{Prg}']/tablas[0]['90 s'], 2)
tablas[0]


elimino sobrantes [1]

In [ ]:
columnas_eliminar1=['País','Posc','Edad','PJ','Titular','Mín','90 s', 'GC','DaPC','PG','PE','PP','PaC','TPint','PD','PD.1','PC','Partidos','Salvadas']
tablas[1]=tablas[1].drop(columnas_eliminar1, axis=1)

elimino sobrantes tabla porteros avanzados[2]

In [ ]:
columnas_eliminar2=['País','Posc','Edad','90 s','GC','GC.1','PD','TL','TE','GC','Partidos','PSxG','PSxG+/-','PSxG','Cmp','Int.','% Cmp','Int.',	'TI',
                    'Opp','Stp','Núm. de OPA']
tablas[2]=tablas[2].drop(columnas_eliminar2, axis=1)
tablas[2]=md.drop_duplicates_columns(tablas[2],'Long. prom.',1)


elimino sobrantes tabla tiros [3]

In [ ]:
tablas[3]['Dis']=round(tablas[3]['Dis']/tablas[3]['90 s'], 2)
tablas[3] = tablas[3].loc[:, ['Jugador','Dis']]


elimino sobrantes tabla pases [4]

In [ ]:

duplicados_a_quitar=['Int.','% Cmp']
for duplicado in duplicados_a_quitar:
    tablas[4]=md.drop_duplicates_columns(tablas[4],duplicado,0)
tablas[4]['Int.']=round(tablas[4]['Int.']/tablas[4]['90 s'],2)
tablas[4] = tablas[4].loc[:, ['Jugador','Int.','% Cmp']]

elimino sobrantes tabla [5]

elimino sobrantes tabla Creación de goles y tiros [5]

In [ ]:
tablas[5]['ACT']=round(tablas[5]['ACT']/tablas[5]['90 s'],2)
tablas[5] = tablas[5].loc[:, ['Jugador','ACT',]]
tablas[5]

tabla Acciones defensivas[6]

In [ ]:
tablas[6]=md.drop_duplicates_columns(tablas[6],'Tkl',0)
tablas[6]['Tkl']=round(tablas[6]['Tkl']/tablas[6]['90 s'], 2)
tablas[6]['Bloqueos']=round(tablas[6]['Bloqueos']/tablas[6]['90 s'], 2)
tablas[6]['Int']=round(tablas[6]['Int']/tablas[6]['90 s'], 2)
tablas[6]['Desp.']=round(tablas[6]['Desp.']/tablas[6]['90 s'], 2)
tablas[6] = tablas[6].loc[:, ['Jugador','Desp.','Int','Bloqueos','Tkl']]



In [ ]:

tablas[6]

In [ ]:
tablas[7]['Ataq. pen.']=round(tablas[7]['Ataq. pen.']/tablas[7]['90 s'], 2)
tablas[7]['Toques']=round(tablas[7]['Toques']/tablas[7]['90 s'], 2)
tablas[7]['Succ']=round(tablas[7]['Succ']/tablas[7]['90 s'], 2)
tablas[7] = tablas[7].loc[:, ['Jugador','Ataq. pen.','Toques','Succ']]
tablas[7]




tabla Estadísticas diversas [8]

In [ ]:
tablas[8]

In [ ]:
tablas[8]['Ganados']=round(tablas[8]['Ganados']/tablas[8]['90 s'], 2)
tablas[8] = tablas[8].loc[:, ['Jugador','Ganados']]
tablas[8]

## bloque 3: unificador de tablas

In [ ]:
temp=tablas[0]
i=0
while i < 8:
    resultado = pd.merge(temp,tablas[i+1], on='Jugador', how='outer')
    temp=resultado
    i+=1

resultado

In [ ]:
resultado.columns

In [16]:
codigo='b8fd03ef'

## bloque 4: asignador de codigo

In [22]:
allteams=pd.read_csv('All_teams.csv')
resultado['Jugador']=int(allteams[allteams['id Web']==f'{codigo}']['id Local'].iloc[0])
resultado


C:\Users\ARTUROS\AppData\Local\Temp\ipykernel_9272\592938234.py:2: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  equipitoBrandy['Jugador']=int(allteams[allteams['id Web']==f'{codigo}']['id Local'])


,Jugador,Posc,Edad,90 s,PrgC,PrgP,PrgR,Ast,G+A-TP,xG+xAG,...,% Cmp,ACT,Desp.,Int,Bloqueos,Tkl,Ataq. pen.,Toques,Succ,Ganados
0,12,CC,26,49.6,1.98,7.00,0.87,0.14,0.22,0.23,...,91.3,2.24,1.05,1.05,1.03,1.83,1.53,88.00,0.69,1.75
1,12,PO,28,47.5,0.00,0.00,0.00,0.02,0.02,0.01,...,85.5,0.04,0.32,0.02,0.00,0.06,0.00,35.28,0.02,0.17
2,12,DL,22,45.7,0.96,1.53,3.98,0.20,1.16,1.13,...,72.4,2.01,0.37,0.11,0.37,0.09,6.04,21.23,0.50,1.47
3,12,CC,31,42.7,1.80,5.76,3.91,0.14,0.40,0.38,...,87.7,2.93,0.44,0.77,0.61,0.98,2.79,62.20,0.63,0.66
4,12,DF,27,42.4,0.80,3.77,0.59,0.02,0.05,0.08,...,92.7,1.01,1.25,0.64,0.97,1.49,1.11,74.20,0.17,0.78
5,12,CC,31,40.7,3.19,7.25,7.44,0.69,0.93,0.69,...,72.2,5.36,0.37,0.37,0.76,0.91,2.83,57.96,1.03,0.34
6,12,"DL,CC",27,40.1,3.32,4.64,6.98,0.15,0.32,0.23,...,86.2,2.82,0.27,0.75,0.85,1.57,3.39,62.37,1.35,0.17
7,12,DL,26,38.6,5.39,3.65,11.01,0.28,0.41,0.47,...,84.4,4.12,0.39,0.36,0.70,0.85,5.08,49.92,1.84,0.47
8,12,DF,25,38.5,0.60,3.58,0.13,0.00,0.03,0.11,...,93.3,0.68,2.10,0.75,0.91,1.22,1.01,85.53,0.29,1.71
9,12,"DL,CC",31,34.0,3.03,2.68,8.97,0.38,0.71,0.66,...,83.0,4.38,0.18,0.38,0.82,1.15,4.03,52.21,1.09,0.29


## bloque 5: Agrupacion de resultados

In [18]:
resultado=resultado.drop('Posc', axis=1)
resultado=resultado.groupby('Jugador').mean().reset_index()
resultado=resultado.rename(columns={'Jugador':'Equipo id'})

12    12
Name: id Local, dtype: int64